In [1]:
import os
from dotenv import load_dotenv
from langchain import hub
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain_community.agent_toolkits.polygon.toolkit import PolygonToolkit
from langchain.agents import AgentExecutor, create_openai_functions_agent, create_tool_calling_agent, tool, create_react_agent



In [7]:
load_dotenv()
os.environ["POLYGON_API_KEY"] = os.getenv("POLYGON_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


# azure key 
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")  # type: ignore
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT") # type: ignore
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = 'gpt-4o-test-app' # type: ignore
os.environ["AZURE_OPENAI_API_VERSION"] = '2024-02-01' # type: ignore

# Initialize the Polygon API wrapper
polygon_api_wrapper = PolygonAPIWrapper()
toolkit = PolygonToolkit.from_polygon_api_wrapper(polygon_api_wrapper)

In [8]:
toolkit.get_tools()

[PolygonAggregates(api_wrapper=PolygonAPIWrapper(polygon_api_key='GsrM4UqmCIk4iqIWEIyfx_jyrkHkGo2a')),
 PolygonLastQuote(api_wrapper=PolygonAPIWrapper(polygon_api_key='GsrM4UqmCIk4iqIWEIyfx_jyrkHkGo2a')),
 PolygonTickerNews(api_wrapper=PolygonAPIWrapper(polygon_api_key='GsrM4UqmCIk4iqIWEIyfx_jyrkHkGo2a')),
 PolygonFinancials(api_wrapper=PolygonAPIWrapper(polygon_api_key='GsrM4UqmCIk4iqIWEIyfx_jyrkHkGo2a'))]

In [9]:

llm= AzureChatOpenAI(
                     azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
                     api_version=os.environ["AZURE_OPENAI_API_VERSION"], # type: ignore
                    )
llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BnhBo8sLxCuhhWeApBFjDYHtHaM2D', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, '

In [4]:
# llm = ChatOpenAI(temperature=0, model="gpt-4o")

llm= AzureChatOpenAI(
                     azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
                     api_version=os.environ["AZURE_OPENAI_API_VERSION"], # type: ignore
                    )


instructions = """You are an assistant."""
base_prompt = hub.pull("hwchase17/react")
prompt = base_prompt.partial(human=instructions)
# agent = create_openai_functions_agent(llm, toolkit.get_tools(), prompt)
agent = create_react_agent(llm = llm1,
                           tools=toolkit.get_tools(),
                            prompt=prompt)

c:\Users\mksgh\OneDrive - Vestas Wind Systems A S\Documents\Github\TradingBot-agentic-langchain\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


NameError: name 'llm1' is not defined

In [28]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    verbose=True,
)
message = agent_executor.invoke({"input": "What was yesterdays financial info for AAPL?"})



> Entering new AgentExecutor chain...
Thought: To find yesterday's financial info for AAPL, I need to get the latest financial data available and then filter it to only include data from yesterday.
Action: polygon_financials
Action Input: AAPL[{"start_date": "2024-03-30", "end_date": "2025-03-29", "timeframe": "ttm", "fiscal_period": "TTM", "fiscal_year": "", "cik": "0000320193", "sic": "3571", "tickers": ["AAPL"], "company_name": "APPLE INC", "financials": {"comprehensive_income": {"comprehensive_income_loss_attributable_to_parent": {"value": 99891000000.0, "unit": "USD", "label": "Comprehensive Income/Loss Attributable To Parent", "order": 300}, "comprehensive_income_loss": {"value": 99891000000.0, "unit": "USD", "label": "Comprehensive Income/Loss", "order": 100}, "other_comprehensive_income_loss_attributable_to_parent": {"value": 2597000000.0, "unit": "USD", "label": "Other Comprehensive Income/Loss Attributable To Parent", "order": 600}, "comprehensive_income_loss_attributable_t

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `llama-3.1-8b-instant` in organization `org_01jq0mrxt0e5saq3he6apqtt3w` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 19403, please reduce your message size and try again. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [26]:
message

{'input': 'What was yesterdays financial info for AAPL?',
 'output': "Apple's financial summary for the latest period is:\n- Revenue: $400.366 billion\n- Net Income: $97.294 billion\n- Gross Profit: $186.699 billion\n- Total Assets: $331.233 billion\n- Total Liabilities: $264.437 billion\n- Equity: $66.796 billion"}

In [8]:
llm1 =  ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [37]:
agent_executor.invoke({"input": "What was yesterdays financial info for AAPL?"})



> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: 39b45bfa********************4527. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain_community.tools.polygon.financials import PolygonFinancials

# 1. Initialize the API wrapper (make sure your POLYGON_API_KEY is set in your environment)
polygon_api_wrapper = PolygonAPIWrapper()

# 2. Create the financials tool
financials_tool = PolygonFinancials(api_wrapper=polygon_api_wrapper)

# 3. Use the tool directly (example: get financials for AAPL)
result = financials_tool.run({"query": "AAPL"})
print(result)

[{"start_date": "2024-03-30", "end_date": "2025-03-29", "timeframe": "ttm", "fiscal_period": "TTM", "fiscal_year": "", "cik": "0000320193", "sic": "3571", "tickers": ["AAPL"], "company_name": "APPLE INC", "financials": {"cash_flow_statement": {"net_cash_flow_from_operating_activities": {"value": 109556000000.0, "unit": "USD", "label": "Net Cash Flow From Operating Activities", "order": 100}, "net_cash_flow_from_operating_activities_continuing": {"value": 109556000000.0, "unit": "USD", "label": "Net Cash Flow From Operating Activities, Continuing", "order": 200}, "net_cash_flow_continuing": {"value": -5759000000.0, "unit": "USD", "label": "Net Cash Flow, Continuing", "order": 1200}, "net_cash_flow_from_financing_activities_continuing": {"value": -129342000000.0, "unit": "USD", "label": "Net Cash Flow From Financing Activities, Continuing", "order": 800}, "net_cash_flow_from_investing_activities_continuing": {"value": 14027000000.0, "unit": "USD", "label": "Net Cash Flow From Investing A

: 